## Modelo de clasificación usando Naive Bayes

In [86]:
# Agregamos las bibliotecas necesarias
import re
import string
import locale
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import numpy as np
from sklearn.model_selection import train_test_split # Biblioteca usada para partir el modelo en grupos de entrenamiento y prueba

Agregamos el dataframe en formato de Document-Term Matrix(DTM), para trabajarlo y usarlo para el modelo de clasificación, además del dataframe con la corpora para saber a que clase pertenecen cada texto.

In [6]:
df_corpus = pd.read_pickle("df_corpus.pkl")
data_dtm = pd.read_pickle("df_DTM.pkl")

Verificamos cuales son las dimenciones de nuestros dataframes:

In [20]:
df_corpus.shape

(104, 4)

In [97]:
df_corpus['categoria'].value_counts()

mantiene      72
incrementa    18
disminuye     14
Name: categoria, dtype: int64

In [23]:
data_dtm.shape

(104, 2865)

Agregamos la columna de "Categoría" a df_corpus (que habíamos pasado por alto del ejercicio inicial de EDA).

In [10]:
df_corpus['categoria'] = df_corpus['text'].str.extract("(?P<category>mantiene|incrementa|disminuye)")
df_corpus.head()

,text,url,contenido,categoria
date,,,,
2008-02-15,El objetivo para la Tasa de Inter�s Interbanca...,https://www.banxico.org.mx/publicaciones-y-pre...,de febrero de comunicado de prensa anunc...,mantiene
2008-03-14,El objetivo para la Tasa de Inter�s Interbanca...,https://www.banxico.org.mx/publicaciones-y-pre...,de marzo de comunicado de prensa anuncio ...,mantiene
2008-04-18,El objetivo para la Tasa de Inter�s Interbanca...,https://www.banxico.org.mx/publicaciones-y-pre...,de abril de comunicado de prensa anuncio de ...,mantiene
2008-05-16,El objetivo para la Tasa de Inter�s Interbanca...,https://www.banxico.org.mx/publicaciones-y-pre...,de mayo de comunicado de prensa anuncio de ...,mantiene
2008-06-20,El objetivo para la Tasa de Inter�s Interbanca...,https://www.banxico.org.mx/publicaciones-y-pre...,de junio de comunicado de prensa anuncio de...,incrementa


Partimos el conjunto de datos del dataframe data_dtm en dos, uno para entrenamiento y otro para probar la eficiencia del modelo, usando una proporción de 60% / 40% respectivamente. Además hacemos que nuestras etiquetas en "categoria" sean nuestras clases objetivo a clasificar.

In [73]:
X_train, X_test, y_train, y_test = train_test_split(data_dtm, df_corpus['categoria'], test_size=0.4, random_state=1)

Verificamos visualmente a los conjuntos usados para el entrenamiento:

In [74]:
X_train

,abastecimiento,abasto,abatimiento,abatir,abierto,abril,abriljunio,abrió,abrupta,absorber,...,éstos,índice,índices,índole,última,últimas,último,últimos,ﬁancladasﬂ,ﬁanclajeﬂ
date,,,,,,,,,,,,,,,,,,,,,
2016-08-11,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
2010-03-19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2011-04-15,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
2018-04-12,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2015-10-29,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-05-05,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2008-11-28,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
2016-02-04,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [75]:
print(X_train.shape)

(62, 2865)


In [76]:
X_test

,abastecimiento,abasto,abatimiento,abatir,abierto,abril,abriljunio,abrió,abrupta,absorber,...,éstos,índice,índices,índole,última,últimas,último,últimos,ﬁancladasﬂ,ﬁanclajeﬂ
date,,,,,,,,,,,,,,,,,,,,,
2018-12-20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,1,0,0
2011-05-27,0,0,0,0,0,1,0,0,0,0,...,1,0,0,0,1,1,1,0,0,0
2017-02-09,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
2014-06-06,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
2011-10-14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
2011-12-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2013-09-06,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017-08-10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2010-11-26,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,0,0


In [77]:
print(X_test.shape)

(42, 2865)


In [78]:
y_train

date
2016-08-11     mantiene
2010-03-19     mantiene
2011-04-15     mantiene
2018-04-12     mantiene
2015-10-29     mantiene
                ...    
2016-05-05     mantiene
2008-11-28     mantiene
2016-02-04     mantiene
2009-03-20    disminuye
2011-08-26     mantiene
Name: categoria, Length: 62, dtype: object

In [79]:
print(y_test)

date
2018-12-20    incrementa
2011-05-27      mantiene
2017-02-09    incrementa
2014-06-06     disminuye
2011-10-14      mantiene
2011-12-02      mantiene
2013-09-06     disminuye
2017-08-10      mantiene
2010-11-26      mantiene
2014-01-31      mantiene
2019-03-28      mantiene
2012-10-26      mantiene
2018-05-17      mantiene
2014-04-25      mantiene
2011-03-04      mantiene
2017-12-14    incrementa
2019-11-14     disminuye
2016-12-15    incrementa
2009-08-21      mantiene
2019-08-15     disminuye
2015-06-04      mantiene
2017-03-30    incrementa
2015-04-30      mantiene
2012-07-20      mantiene
2012-04-27      mantiene
2012-01-20      mantiene
2017-05-18    incrementa
2017-09-28      mantiene
2013-01-18      mantiene
2011-07-08      mantiene
2009-01-16     disminuye
2008-04-18      mantiene
2011-01-21      mantiene
2015-09-21      mantiene
2009-10-16      mantiene
2010-07-16      mantiene
2014-10-31      mantiene
2019-05-16      mantiene
2016-09-29    incrementa
2016-03-18      mant

In [80]:
print(y_test.shape)

(42,)


Model Building and Evaluation

Let's build the Text Classification Model using TF-IDF.

First, import the MultinomialNB module and create a Multinomial Naive Bayes classifier object using MultinomialNB() function.

Then, fit your model on a train set using fit() and perform prediction on the test set using predict().

In [81]:
from sklearn.naive_bayes import MultinomialNB
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Generation Using Multinomial Naive Bayes
clf = MultinomialNB().fit(X_train, y_train)
predicted= clf.predict(X_test)
print("MultinomialNB Accuracy:",metrics.accuracy_score(y_test, predicted))

MultinomialNB Accuracy: 0.6904761904761905


En este caso, revisamos la matriz de confusión para ver otros parámetros de prueba de precisión del modelo:

In [87]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, predicted)

array([[ 0,  0,  6],
       [ 0,  0,  7],
       [ 0,  0, 29]], dtype=int64)

In [88]:
y_test.value_counts()

mantiene      29
incrementa     7
disminuye      6
Name: categoria, dtype: int64

In [89]:
unique, counts = np.unique(predicted, return_counts=True)
dict(zip(unique, counts))

{'mantiene': 42}

Normalizamos el conjunto de datos inicial y lo pasamos nuevamente al modelo

In [90]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer()
text_tf= tf.fit_transform(df_corpus['contenido'])

In [91]:
X_train, X_test, y_train, y_test = train_test_split(text_tf, df_corpus['categoria'], test_size=0.4, random_state=123)

Probamos la eficiencia del modelo normalizado:

In [92]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
# Model Generation Using Multinomial Naive Bayes
clf = MultinomialNB().fit(X_train, y_train)
predicted= clf.predict(X_test)
print("MultinomialNB Accuracy:",metrics.accuracy_score(y_test, predicted))

MultinomialNB Accuracy: 0.6904761904761905


In [93]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, predicted)

array([[ 0,  0,  7],
       [ 0,  0,  6],
       [ 0,  0, 29]], dtype=int64)

In [94]:
y_test.value_counts()

mantiene      29
disminuye      7
incrementa     6
Name: categoria, dtype: int64

In [95]:
unique, counts = np.unique(predicted, return_counts=True)
dict(zip(unique, counts))

{'mantiene': 42}

Como se puede notar, el modelo no logra clasificar con exactitud las clases de "disminuye" y "aumenta".